# Goals
    _x_ Install Tensorflow 
    _x_ Install pymongo
    _x_ Read from pymongo
    ___ 
    ___ 

# Installs

In [6]:
# import sys

# !pip install --prefix {sys.prefix} tensorflow

In [7]:
import sys

!pip install --prefix {sys.prefix} pymongo

# Imports

In [42]:
import pickle
from PIL import Image 
import pandas as pd
import pymongo
from sklearn.model_selection import train_test_split
from sklearn import tree
import tensorflow as tf
# smart phrone: ssd_mobilenet
# laptop: rcnn

# Connect to MongoDB

In [22]:
user = ""
pswd = ""

client = pymongo.MongoClient("mongodb+srv://{}:{}@cluster0.ihx5p.mongodb.net/<dbname>?retryWrites=true&w=majority".format(user, pswd))
db = client["DATA_603"]
collection = db["test_images"]

# Test Connection

In [18]:
def check_db_collection(race="African American"):
    count = 0
    cursor = collection.find({"race": race})
    for document in cursor:
        # num = document["_id"]
        # print(num)
        # num = num.split("_")[-1]
        # print(num)
        # if int(num) % 10 == 0:
        print(document["_id"])
        img = pickle.loads( document["numpy_arr"] )
        Image.fromarray(img, 'RGB').show(title=document["_id"])
        count += 1 
        if count == 10:
            return 
    
# check_db_collection()

African_American_0
African_American_1
African_American_2
African_American_3
African_American_4
African_American_5
African_American_6
African_American_7
African_American_8
African_American_9


# Hold images in a container

In [75]:
print(collection.count_documents({}))
%time df = pd.DataFrame(collection.find({})[0:100])
%time df["numpy_arr"] = pickle.loads( df["numpy_arr"] )
%time df["numpy_arr"] = df["numpy_arr"].apply(lambda x: pickle.loads(x))
%time df["numpy_arr"] = df["numpy_arr"].apply(lambda x: Image.fromarray(x, 'RGB'))
%time df

568
CPU times: user 418 ms, sys: 448 ms, total: 866 ms
Wall time: 13 s


TypeError: a bytes-like object is required, not 'Series'

CPU times: user 81.1 ms, sys: 27.3 ms, total: 108 ms
Wall time: 111 ms
CPU times: user 27.4 ms, sys: 17.2 ms, total: 44.5 ms
Wall time: 45.5 ms
CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.2 µs


,_id,source_url,numpy_arr,race
0,African_American_0,https://image.shutterstock.com/image-photo/smi...,<PIL.Image.Image image mode=RGB size=356x280 a...,African American
1,African_American_1,https://image.shutterstock.com/image-photo/you...,<PIL.Image.Image image mode=RGB size=390x280 a...,African American
2,African_American_2,https://image.shutterstock.com/image-photo/bea...,<PIL.Image.Image image mode=RGB size=390x280 a...,African American
3,African_American_3,https://image.shutterstock.com/image-photo/smi...,<PIL.Image.Image image mode=RGB size=390x280 a...,African American
4,African_American_4,https://image.shutterstock.com/image-photo/you...,<PIL.Image.Image image mode=RGB size=390x280 a...,African American
...,...,...,...,...
95,White_35,https://image.shutterstock.com/image-photo/low...,<PIL.Image.Image image mode=RGB size=361x280 a...,White
96,White_36,https://image.shutterstock.com/image-photo/suc...,<PIL.Image.Image image mode=RGB size=390x280 a...,White
97,White_37,https://image.shutterstock.com/image-photo/clo...,<PIL.Image.Image image mode=RGB size=246x280 a...,White
98,White_38,https://image.shutterstock.com/image-photo/bla...,<PIL.Image.Image image mode=RGB size=390x280 a...,White


# Rerformat Images (Grayscale, fixed size)
Ref: https://machinelearningmastery.com/how-to-load-and-manipulate-images-for-deep-learning-in-python-with-pil-pillow/

In [76]:
# Convert the image to grayscale
df["numpy_arr"] = df["numpy_arr"].apply(lambda x: x.convert(mode='L'))
# REsize Image
df["numpy_arr"] = df["numpy_arr"].apply(lambda x: x.resize((150,250)))

CPU times: user 30.3 ms, sys: 11.2 ms, total: 41.6 ms
Wall time: 42.4 ms


In [77]:
X = df.numpy_arr
Y = df.race

X_train, X_test, Y_train, Y_test = train_test_split(X,
                                                    Y,
                                                    test_size=0.2,
                                                    shuffle=True,
                                                    random_state=2020)

In [86]:
temp = pd.DataFrame()

temp["L"] = X.apply(lambda x: x.size[0])
temp["W"] = X.apply(lambda x: x.size[1])

temp.describe()

,L,W
count,100.00000,100.0
mean,358.25000,280.0
std,71.85625,0.0
min,173.00000,280.0
25%,387.75000,280.0
50%,390.00000,280.0
75%,390.00000,280.0
max,446.00000,280.0


# Using sklearn 

In [87]:
model = tree.DecisionTreeClassifier()
model = model.fit(X_train, Y_train)

TypeError: float() argument must be a string or a number, not 'Image'